# HW2 Problem 5: Optimal Control of CSTR

### Optimization model
Let $\mathcal{K}_x = \{0,1,2,3\}$ and $\mathcal{K}_u = \{0,1,2\}$

Objective: $$\min \sum_{k\in \mathcal{K}_x} s_1(k) + s_3(k)$$

Subject to: 
$$
\begin{aligned}
    x(0) &= \left[ -0.03,0,0.3 \right]^T\\
    x(k+1) &= A x(k) + B u(k) \qquad &\forall k\in\mathcal{K}_u\\
    y(k) &= C\,x(k) \qquad &\forall k\in\mathcal{K}_x\\
    s_1(k)&\ge \pm x_1(k) \qquad &\forall k\in\mathcal{K}_x\\ 
    s_3(k)&\ge \pm x_3(k) \qquad &\forall k\in\mathcal{K}_x\\
    x_{min} &\le x(k) \le x_{max} \qquad &\forall k\in\mathcal{K}_x\\
    u_{min} &\le u(k) \le u_{max} \qquad &\forall k\in\mathcal{K}_u
\end{aligned}
$$

In [15]:
import pyomo.environ as pyo

A = {
    (1,1): 0.2681, (1,2): -0.00338, (1,3): -0.00728,
    (2,1): 9.703,  (2,2): 0.3279,   (2,3): -25.44,
    (3,1): 0.0,    (3,2): 0.0,      (3,3): 1.0
}

B = {
    (1,1): -0.00537, (1,2): 0.1655,
    (2,1):  1.297,   (2,2): 97.91,
    (3,1):  0.0,     (3,2): -6.637
}

x0 = {1: -0.03, 2: 0.0, 3: 0.3}

x_min = {1: -0.05, 2: -5.0, 3: -0.5}
x_max = {1:  0.05, 2:  5.0, 3:  0.5}

u_min = {1: -10.0, 2: -0.05}
u_max = {1:  10.0, 2:  0.05}

Kx = [0, 1, 2, 3]   # state/output indices
Ku = [0, 1, 2]      # input indices (moves)

In [16]:
# Initialize model
m = pyo.ConcreteModel()

# Initialize sets and parameters
m.I = pyo.Set(initialize=[1, 2, 3])     # states
m.J = pyo.Set(initialize=[1, 2])        # inputs
m.Kx = pyo.Set(initialize=Kx, ordered=True)
m.Ku = pyo.Set(initialize=Ku, ordered=True)
m.AbsIdx = pyo.Set(initialize=[1, 3])   # we penalize |x1| and |x3|

m.A = pyo.Param(m.I, m.I, initialize=A)
m.B = pyo.Param(m.I, m.J, initialize=B)

# Initialize constraints
def x_bounds(m, k, i):
    return (x_min[i], x_max[i])

def u_bounds(m, k, j):
    return (u_min[j], u_max[j])

m.x = pyo.Var(m.Kx, m.I, bounds=x_bounds)          # state trajectory
m.u = pyo.Var(m.Ku, m.J, bounds=u_bounds)          # input moves
m.s = pyo.Var(m.Kx, m.AbsIdx, domain=pyo.NonNegativeReals)  # abs auxiliaries

# Initial condition (fix x(0) = x0)
for i in m.I:
    m.x[0, i].fix(x0[i])

# Dynamics: x(k+1) = A x(k) + B u(k)  for k=0,1,2
def dyn_rule(m, k, i):
    return m.x[k+1, i] == sum(m.A[i, j] * m.x[k, j] for j in m.I) + sum(m.B[i, j] * m.u[k, j] for j in m.J)
m.dyn = pyo.Constraint(m.Ku, m.I, rule=dyn_rule)

# Absolute value linearization:
# s1(k) >=  x1(k), s1(k) >= -x1(k)
# s3(k) >=  x3(k), s3(k) >= -x3(k)
def abs_pos_rule(m, k, idx):
    return m.s[k, idx] >= m.x[k, idx]
def abs_neg_rule(m, k, idx):
    return m.s[k, idx] >= -m.x[k, idx]

m.abs_pos = pyo.Constraint(m.Kx, m.AbsIdx, rule=abs_pos_rule)
m.abs_neg = pyo.Constraint(m.Kx, m.AbsIdx, rule=abs_neg_rule)

In [17]:
# Objective
m.obj = pyo.Objective(
    expr=sum(m.s[k, 1] + m.s[k, 3] for k in m.Kx),
    sense=pyo.minimize
)

# Solve
solver = pyo.SolverFactory("gurobi", solver_io='python')
res = solver.solve(m, tee=False)

In [18]:
# Display results
print("\nObjective value =", pyo.value(m.obj))
for k in m.Kx:
    print(f"Time step {k}:")
    for i in m.I:
        print(f"  State x[{i}] = {pyo.value(m.x[k,i])}")
    if k < max(m.Ku):
        for i in m.J:
            print(f"  Input u[{i}] = {pyo.value(m.u[k,i])}")


Objective value = 0.32999999999999996
Time step 0:
  State x[1] = -0.03
  State x[2] = 0.0
  State x[3] = 0.3
  Input u[1] = -0.5113986008688389
  Input u[2] = 0.04520114509567576
Time step 1:
  State x[1] = 0.0
  State x[2] = -4.160729869009269
  State x[3] = 0.0
  Input u[1] = 2.618857906378274
  Input u[2] = 0.0
Time step 2:
  State x[1] = 0.0
  State x[2] = 2.032355380524481
  State x[3] = 0.0
Time step 3:
  State x[1] = 0.0
  State x[2] = -0.9927268827308744
  State x[3] = 0.0
